### Note:
- The test cases here only check that type of output that you are returning is correct. They don't check whether the values that you are returning are actually correct. 

## Perceptron Algorithm
In this quiz, we are going to implement the perceptron learning algorithm which was discussed in class. (Refer to reference videos of Prof Behera)

In [ ]:
import numpy as np

### Generate data
Generate random points in two circles like this. <br>
<img src = "https://drive.google.com/uc?id=17doTxsW980SXmJwFrmZe93VKG6hzF2hc">

##1

### Random points in circle
Write a function that takes as parameters center - $c$, radius - $r$ and number ($n$) of random points and returns an array of $n$ random points in circle with center $c$ and radius $r$.

In [ ]:
import random
def rand_points_in_circle(c, r, n):
    """
    Inputs:
        c: tuple of floats of length 2, co-ordinates of the center of circle
        r: float, radius of circle
        n: int, number of random points to generate
    Outputs:
        points: numpy array of shape (n, 2) with random points from circle 
    """
    circle_x=c[0]
    circle_y=c[1]
    points=np.array([[0.0 for i in range(0,2)]for i in range(0,n)],dtype=float)
    circle_r=r*np.sqrt(random.random())
    for i in range(0,n):
      alpha=2*np.pi*random.random()
      points[i][0]=round((circle_r*np.cos(alpha)+circle_x),4)
      points[i][1]=round((circle_r*np.sin(alpha)+circle_y),4)
    return(points)

In [ ]:
"""Test Cases"""
if __name__ == '__main__':
  assert rand_points_in_circle((2., 2.), 3., 4).shape == (4, 2)
  print('Test passed', '\U0001F44D')

Test passed 👍


##2

### Generate Data
Use the rand_circle function to generate the $+$ and $-$ points. <br>
The $y$ values corresponding to $+$ points will be $+1$ and $-$ points will be $-1$

In [ ]:
def gen_data(c1, r1, c2, r2, n):
  """
  This function generates 2n points
    - n random points in circle with center c1 and radius r1 - These are the + points, y value for these points is +1
    - n random points in circle with center c2 and radius r2 - These are the - points, y value for these points is -1

  Inputs:
    c1 - tuple of floats of length 2, co-ordinates of center of circle-1
    r1 - float, radius of circle-1
    c2 - tuple of floats of length 2, co-ordinates of center of circle-2
    r2 -  float, radius of circle-2
    n - int, Number of points in each circle

  Outputs:
    X : numpy array of samples of shape (2n, 2)
    y : numpy array of labels (+1/ -1) of shape (2n, )
  """
  x1=rand_points_in_circle(c1, r1, n)
  x2=rand_points_in_circle(c2, r2, n)
  X=np.vstack([x1,x2])
  y=[1]*n + [-1]*n
  y=np.array(y)
  return X, y

In [ ]:
""" Test Cases """
if __name__ == "__main__":
  X, y = gen_data((1, 1), 2, (10, 10), 4, 10)
  assert X.shape == (20, 2)
  assert y.shape == (20, )
  print('Test passed', '\U0001F44D')

Test passed 👍


##3

### Create Perceptron model
The perceptron model we shall use is defined as follows : <br>
$$
y_{pred} = \text{Sign}(\sum_{i=0}^{2}{w_i.x_i})
$$
where $\text{Sign}$ is the sign function <br> 
$\text{Sign}(x) = 1$ if $x>=0$ <br>
$\text{Sign}(x) = -1$ if $x<0$ <br>
Also, <br>
$x_0 = 1$ and $x_1$ and $x_2$ are the coordinates of the points. <br>
$w_0$, $w_1$ and $w_2$ are parameters of our model.


####Perceptron Model
Write a function that given the values of $\bf w$ and $\bf X$, applies the perceptron model and returns the predictions.

In [ ]:
def perceptron_predict(w, X):
    """
    Inputs:
        w: numpy array of shape (3, ), model weights
        X: numpy array of shape (n, 2), sample points
    
    Outputs:
        y_pred: numpy array of shape (n,) with values +1 / -1 obtained applying perceptron model to X
    """
    y=[]
    for i in range(0,X.shape[0]):
      h=np.dot(X[i],w[1:].T)+w[0]
      y.append(1) if h>=0 else y.append(-1)
    y_pred=np.array(y)
    return y_pred

In [ ]:
"""Test cases"""
if __name__ == '__main__':
  y_pred = perceptron_predict(np.array([1, 2, 3]), np.arange(10).reshape(5, 2))
  assert y_pred.shape == (5, )
  print('Test passed', '\U0001F44D')

Test passed 👍


##4

### Loss
Our loss function is ```number of misclassified points```. <br>
Write a function that takes as inputs $w$, $X$, $y_d$ and returns two values 
 - number of misclassified points, ```n_wrong```
 - a random misclassified point ```x_wrong``` 
 
If number of misclassified points is 0, then ```x_wrong``` should be ```None```

In [ ]:
def misclassified(X, yd, w):
    """
    Inputs:
        X : numpy array of shape (n, 2), sample points
        yd : numpy array of shape (n,), true labels
        w : numpy array of shape (3, ), model weights
    Outputs:
        n_wrong: int, number of misclassified points
        x_wrong: numpy array of shape (2, ) - a misclassified point, None if n_wrong is 0
    """
    x_wrong=np.empty((2))
    n_wrong=0
    y=perceptron_predict(w,X)
    for i in range(0,yd.shape[0]):
      if(yd[i]!=y[i]):
        n_wrong+=1
        x_wrong=X[i]
    if(n_wrong==0):
      x_wrong=None
    return n_wrong, x_wrong

In [ ]:
""" Test Cases"""
if __name__ == '__main__':
  n_wrong, x_wrong = misclassified(np.arange(10).reshape(5, 2), np.array([-1]*5), np.ones(3))
  assert n_wrong > 0
  assert x_wrong.shape == (2, )
  print('Test passed', '\U0001F44D')

Test passed 👍


##5

### Training
Apply perceptron learning algorithm and find parameters $w$ which perfectly separate the $+$ and $-$ points. <br>
We shall use the following learning algorithm - <br>
<br>
- Start with random weights.
- Do the following steps till all points are correctly classified:
  - Pick a misclassified point $(x_i, yd_i)$ {$yd_i$ is true label corresponding to $x_i$}
  - Update weights:
$$
w_{new} = w_{old} + yd_i.x_i 
$$

In [ ]:
def perceptron_train(X, y):
  """
  This function trains the perceptron model starting from random weights.

  Inputs:
    X: numpy array of shape (n, 2) - sample points
    y: numpy array of shape (n, ) - true labels
  
  Outputs:
    w: numpy array of shape (3, ) - model parameters after training
  """
  w=np.random.rand(3)
  n_wrong,x_wrong=misclassified(X,y,w)
  c=0
  while(n_wrong>0):
    idx=np.where(X==x_wrong)[0][0]
    y_i=y[idx]
    x_i=np.append(1,x_wrong)
    w+=y_i*x_i
    n_wrong,x_wrong=misclassified(X,y,w)
    c+=1
  #print(n_wrong,c) THIS SHOULD GIVE THE ANS AS 0,#ITERATIONS
  return w